<a href="https://colab.research.google.com/github/OmarAfify10/wiki_m/blob/main/wiki_xml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import chardet

with open('/content/wiki_main.xml', 'rb') as f:
    raw = f.read(10000)  # read first 10k bytes
    result = chardet.detect(raw)
    print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [14]:
with open('/content/wiki_main.xml', 'rb') as f:
    data = f.read(1000000)  # read 1MB
    print(chardet.detect(data))


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [ ]:
#with open('wiki_main.xml', 'rb') as f:
    #raw = f.read()

# Attempt to fix broken utf-8 sequences wrongly decoded as latin1
#fixed_text = raw.decode('Latin1').encode('utf-8').decode('utf-8')

#print(fixed_text[:22500])


In [15]:
import xml.etree.ElementTree as ET

try:
    ET.parse('wiki_main.xml')
    print("XML is valid.")
except ET.ParseError as e:
    print("Invalid XML:", e)


XML is valid.


In [ ]:
#from xml.etree.ElementTree import iterparse, ElementTree, Element

#input_file = "wiki_main.xml"
#output_file = "cleaned.xml"

#context = iterparse(input_file, events=("start", "end"))
#_, root = next(context)  # get root element

#valid_elements = []

#try:
 #   for event, elem in context:
  #      if event == "end":
   #         valid_elements.append(elem)
    #        root.clear()
#except Exception as e:
 #   print("Stopped parsing due to error:", e)

# Save valid elements into a new XML file
#new_root = Element(root.tag)
#new_root.extend(valid_elements)

#tree = ElementTree(new_root)
#tree.write(output_file, encoding='utf-8', xml_declaration=True)
#print(f"Saved cleaned XML to {output_file}")


Saved cleaned XML to cleaned.xml


In [16]:
import xml.etree.ElementTree as ET
import re

input_file = "wiki_main.xml"
output_file = "wiki_cleaned_text_only.xml"

# VIN and email patterns
vin_pattern = re.compile(r'\b[A-HJ-NPR-Z0-9]{17}\b')
email_pattern = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', re.IGNORECASE)

# Tags whose content should be fully excluded
excluded_tags = {"username", "ip", "id", "contributor", "sha1", "origin", "model", "format"}

# Start parsing
context = ET.iterparse(input_file, events=("end",))
_, root = next(context)

with open(output_file, "w", encoding="utf-8") as f:
    for event, elem in context:
        tag_name = elem.tag.split('}')[-1].lower()

        # Skip both tag and its content if it's in the exclusion list
        if tag_name in excluded_tags:
            elem.clear()
            continue

        # Get text and clean it
        text = "".join(elem.itertext()).strip()
        if not text:
            elem.clear()
            continue

        # Remove VINs and emails
        text = vin_pattern.sub("", text)
        text = email_pattern.sub("", text)

        # Write cleaned visible text only
        f.write(text + "\n")

        elem.clear()

print(f"✅ Fully cleaned plain-text file saved as: {output_file}")


✅ Fully cleaned plain-text file saved as: wiki_cleaned_text_only.xml


In [ ]:
import xml.etree.ElementTree as ET
import re

input_file = "wiki_main.xml"
output_file = "wiki_cleaned_text_only.xml"

# VIN and email patterns
vin_pattern = re.compile(r'\b[A-HJ-NPR-Z0-9]{17}\b')
email_pattern = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', re.IGNORECASE)

# Parse the file
context = ET.iterparse(input_file, events=("end",))
_, root = next(context)

with open(output_file, "w", encoding="utf-8") as f:
    for event, elem in context:
        # Get all text content from this element
        text = "".join(elem.itertext()).strip()

        # Skip empty content
        if not text:
            elem.clear()
            continue

        # Remove sensitive patterns
        text = vin_pattern.sub("", text)
        text = email_pattern.sub("", text)

        # Write clean text (no XML tags)
        f.write(text + "\n")

        # Clear memory
        elem.clear()

print(f"✅ Cleaned plain-text XML written to: {output_file}")




✅ Cleaned plain-text XML written to: wiki_cleaned_text_only.xml


In [ ]:
import xml.etree.ElementTree as ET

# Load the full XML file
#tree = ET.parse('cleaned.xml')
tree = ET.parse('wiki_main.xml')
root = tree.getroot()

# Get all child elements (assuming you want to split these)
children = list(root)

# Split point (halfway)
mid = len(children) // 2

# Helper function to write part of the tree
def write_partial_xml(elements, filename):
    partial_root = ET.Element(root.tag, root.attrib)
    partial_root.extend(elements)
    partial_tree = ET.ElementTree(partial_root)
    partial_tree.write(filename, encoding='utf-8', xml_declaration=True)

# Write the two parts
write_partial_xml(children[:mid], 'part1.xml')
write_partial_xml(children[mid:], 'part2.xml')

print("XML file successfully split into 'part1.xml' and 'part2.xml'")

XML file successfully split into 'part1.xml' and 'part2.xml'
